In [1]:
import torch
# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"Current GPU: {torch.cuda.get_device_name()}")

Using device: cuda
Current GPU: NVIDIA GeForce RTX 3060


In [7]:
import pandas as pd
import torch
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau


# 데이터셋 로드 및 전처리
def load_and_preprocess_data(data_path):
    data = pd.read_csv(data_path)
    vectorizer = CountVectorizer()
    data['cleaned_ingredients'] = data['ingredients'].apply(eval).apply(
        lambda x: [''.join(ingredient.lower().split()) for ingredient in x]
    )
    data['ingredient_text'] = data['cleaned_ingredients'].apply(' '.join)
    ingredient_vectors = vectorizer.fit_transform(data['ingredient_text']).toarray()

    # 데이터 정규화
    scaler = StandardScaler()
    ingredient_vectors = scaler.fit_transform(ingredient_vectors)

    cocktail_labels = {name: idx for idx, name in enumerate(data['name'].unique())}
    data['labels'] = data['name'].map(cocktail_labels)

    # 벡터라이저 저장
    with open("model/vectorizer.pkl", "wb") as f:
        pickle.dump(vectorizer, f)

    return data, ingredient_vectors, cocktail_labels


# 데이터셋 클래스 정의
class MultiLabelDataset(Dataset):
    def __init__(self, ingredient_vectors, labels, num_classes):
        self.X = torch.tensor(ingredient_vectors, dtype=torch.float32)
        self.y = torch.nn.functional.one_hot(
            torch.tensor(labels, dtype=torch.long), num_classes=num_classes
        ).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# 더 깊은 네트워크 모델 정의
class DeeperMultiLabelClassifier(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(DeeperMultiLabelClassifier, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, 2048)
        self.bn1 = torch.nn.BatchNorm1d(2048)
        self.relu1 = torch.nn.ReLU()
        self.dropout1 = torch.nn.Dropout(0.5)

        self.fc2 = torch.nn.Linear(2048, 1024)
        self.bn2 = torch.nn.BatchNorm1d(1024)
        self.relu2 = torch.nn.ReLU()
        self.dropout2 = torch.nn.Dropout(0.5)

        self.fc3 = torch.nn.Linear(1024, 512)
        self.bn3 = torch.nn.BatchNorm1d(512)
        self.relu3 = torch.nn.ReLU()
        self.dropout3 = torch.nn.Dropout(0.5)

        self.fc4 = torch.nn.Linear(512, 256)
        self.bn4 = torch.nn.BatchNorm1d(256)
        self.relu4 = torch.nn.ReLU()
        self.dropout4 = torch.nn.Dropout(0.5)

        self.fc5 = torch.nn.Linear(256, 128)
        self.bn5 = torch.nn.BatchNorm1d(128)
        self.relu5 = torch.nn.ReLU()
        self.dropout5 = torch.nn.Dropout(0.5)

        self.fc6 = torch.nn.Linear(128, num_classes)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))
        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.dropout4(self.relu4(self.bn4(self.fc4(x))))
        x = self.dropout5(self.relu5(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x


# 손실 함수 정의
class LabelSmoothingLoss(torch.nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing

    def forward(self, inputs, targets):
        targets = targets * (1 - self.smoothing) + self.smoothing / targets.size(1)
        return torch.nn.BCELoss()(inputs, targets)


# 학습 코드
def train_deeper_model(data_path, model_save_path, epochs=300, batch_size=16):
    data, ingredient_vectors, cocktail_labels = load_and_preprocess_data(data_path)
    dataset = MultiLabelDataset(ingredient_vectors, data['labels'].tolist(), num_classes=len(cocktail_labels))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = DeeperMultiLabelClassifier(input_size=ingredient_vectors.shape[1], num_classes=len(cocktail_labels))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
    criterion = LabelSmoothingLoss(smoothing=0.1)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for inputs, targets in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        scheduler.step(total_loss)

        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(dataloader):.4f}")

    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to '{model_save_path}'.")


# 실행
if __name__ == "__main__":
    train_deeper_model(
        data_path="data/final_cocktails.csv",
        model_save_path="model/deeper_multi_label_model.pt",
        epochs=300,
        batch_size=16
    )


d:\project\OSS_Project\AI\myenv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [10/300], Loss: 0.0222
Epoch [20/300], Loss: 0.0184
Epoch [30/300], Loss: 0.0173
Epoch [40/300], Loss: 0.0169
Epoch [50/300], Loss: 0.0163
Epoch [60/300], Loss: 0.0159
Epoch [70/300], Loss: 0.0155
Epoch [80/300], Loss: 0.0148
Epoch [90/300], Loss: 0.0139
Epoch [100/300], Loss: 0.0133
Epoch [110/300], Loss: 0.0127
Epoch [120/300], Loss: 0.0116
Epoch [130/300], Loss: 0.0116
Epoch [140/300], Loss: 0.0109
Epoch [150/300], Loss: 0.0103
Epoch [160/300], Loss: 0.0102
Epoch [170/300], Loss: 0.0099
Epoch [180/300], Loss: 0.0095
Epoch [190/300], Loss: 0.0093
Epoch [200/300], Loss: 0.0089
Epoch [210/300], Loss: 0.0088
Epoch [220/300], Loss: 0.0085
Epoch [230/300], Loss: 0.0083
Epoch [240/300], Loss: 0.0083
Epoch [250/300], Loss: 0.0083
Epoch [260/300], Loss: 0.0081
Epoch [270/300], Loss: 0.0081
Epoch [280/300], Loss: 0.0080
Epoch [290/300], Loss: 0.0083
Epoch [300/300], Loss: 0.0078
Model saved to 'model/deeper_multi_label_model.pt'.


In [13]:
import torch
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


# 모델 정의 (학습 시 사용한 동일한 모델 구조)
class DeepMultiLabelClassifier(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(DeepMultiLabelClassifier, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, 1024)
        self.bn1 = torch.nn.BatchNorm1d(1024)
        self.relu1 = torch.nn.ReLU()
        self.dropout1 = torch.nn.Dropout(0.5)

        self.fc2 = torch.nn.Linear(1024, 512)
        self.bn2 = torch.nn.BatchNorm1d(512)
        self.relu2 = torch.nn.ReLU()
        self.dropout2 = torch.nn.Dropout(0.5)

        self.fc3 = torch.nn.Linear(512, 256)
        self.bn3 = torch.nn.BatchNorm1d(256)
        self.relu3 = torch.nn.ReLU()
        self.dropout3 = torch.nn.Dropout(0.5)

        self.fc4 = torch.nn.Linear(256, 128)
        self.bn4 = torch.nn.BatchNorm1d(128)
        self.relu4 = torch.nn.ReLU()
        self.dropout4 = torch.nn.Dropout(0.5)

        self.fc5 = torch.nn.Linear(128, num_classes)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))
        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.dropout4(self.relu4(self.bn4(self.fc4(x))))
        x = self.sigmoid(self.fc5(x))
        return x


# 테스트 함수 정의
def test_model(model_path, vectorizer_path, data_path, test_ingredients, top_n=5):
    """
    학습된 모델을 사용하여 테스트 입력에 대한 추천 결과를 출력합니다.
    
    Args:
        model_path (str): 학습된 모델 파일 경로
        vectorizer_path (str): 학습된 CountVectorizer 파일 경로
        data_path (str): 데이터셋 파일 경로
        test_ingredients (list): 테스트 입력 재료 리스트
        top_n (int): 출력할 추천 결과의 개수
    """
    # 벡터라이저 로드
    with open(vectorizer_path, "rb") as f:
        vectorizer = pickle.load(f)

    # 데이터셋 로드
    data = pd.read_csv(data_path)
    cocktail_labels = {name: idx for idx, name in enumerate(data['name'].unique())}
    idx_to_label = {idx: name for name, idx in cocktail_labels.items()}
    num_classes = len(cocktail_labels)

    # 모델 초기화 및 로드
    input_size = len(vectorizer.get_feature_names_out())
    model = DeepMultiLabelClassifier(input_size=input_size, num_classes=num_classes)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # 테스트 입력 처리
    test_vector = vectorizer.transform([' '.join(test_ingredients)]).toarray()
    test_tensor = torch.tensor(test_vector, dtype=torch.float32)

    # 예측 수행
    with torch.no_grad():
        outputs = model(test_tensor).flatten()
        predicted_indices = torch.topk(outputs, k=top_n).indices.numpy()

    # 추천 결과 출력
    recommendations = [(idx_to_label[idx], outputs[idx].item()) for idx in predicted_indices]
    print("Top Recommendations:")
    for name, score in recommendations:
        print(f"{name}: {score:.2f}")


# 실행
if __name__ == "__main__":
    # 테스트 입력 예제
    test_ingredients = ["baileys", "chocolatesyrup", "milk"]

    # 테스트 코드 실행
    test_model(
        model_path="model/deep_multi_label_model.pt",
        vectorizer_path="model/vectorizer.pkl",
        data_path="data/final_cocktails.csv",
        test_ingredients=test_ingredients,
        top_n=5  # 상위 5개 추천 결과 출력
    )


Top Recommendations:
Screwdriver: 0.42
Godmother: 0.33
Lone Tree Cocktail: 0.30
Iced Coffee Fillip: 0.27
Rum Screwdriver: 0.23


C:\Users\user\AppData\Local\Temp\ipykernel_49096\3932997113.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
